In [1]:
!pip install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/b5/61/beed3c4cefad9038d9ff147cdc38168d5c57b5ec2c697e7a52ff32e0bafa/boto3-1.34.40-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.35.0,>=1.34.40 from https://files.pythonhosted.org/packages/4b/ca/f86918789bb290e9fb7573df4d496435c91196440479afc2b803f9d615bd/botocore-1.34.40-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/12/bb/7e7912e18cd558e7880d9b58ffc57300b2c28ffba9882b3a54ba5ce3ebc4/s3transfer-0.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 887.1 kB/s eta 0:00:001m906.0 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.0 MB/s eta 0:00:00m eta 0:00:01
  Attempting uninstall: botocore
    Found existing installation: b

In [1]:
import boto3
import pandas as pd
import psycopg2
import json

# Load AWS Config

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('cluster.config'))

In [4]:
KEY = config.get("AWS","KEY")
SECRET = config.get("AWS","SECRET")


DWH_CLUSTER_TYPE = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH","DWH_DB")
DWH_DB_USER = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME = config.get("DWH","DWH_IAM_ROLE_NAME")

In [6]:
pd.DataFrame({"Param":["DWH_CLUSTER_TYPE","DWH_NUM_NODES","DWH_NODE_TYPE","DWH_CLUSTER_IDENTIFIER","DWH_DB","DWH_DB_USER","DWH_DB_PASSWORD","DWH_PORT","DWH_IAM_ROLE_NAME"],
                "Value":[DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]})

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,my-first-redshift
4,DWH_DB,myfirstdb
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,Password123
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,redshift-s3-access


# Initialize Ec2, S3, IAM, and Redshift instances

In [7]:
ec2 = boto3.resource('ec2',
                        region_name='us-east-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                   )

In [8]:
s3 = boto3.resource('s3',
                   region_name='us-east-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                   )
iam = boto3.client('iam',
                   region_name='us-east-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                   )
redshift = boto3.client('redshift',
                   region_name='us-east-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET
                   )

In [9]:
bucket = s3.Bucket("tickit-items")
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'category_pipe.txt',
 'date2008_pipe.txt',
 'listings_pipe.txt',
 'sales_tab.txt',
 'venue_pipe.txt']

In [10]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [11]:
roleArn

'arn:aws:iam::992382696383:role/redshift-s3-access'

# Create Redshift Cluster

In [12]:
try:
    response = redshift.create_cluster(
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        
        #identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #roles
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)
    

An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists


In [12]:
redshift.describe_clusters(ClusterIdentifier = DWH_CLUSTER_IDENTIFIER)

{'Clusters': [{'ClusterIdentifier': 'my-first-redshift',
   'NodeType': 'dc2.large',
   'ClusterStatus': 'available',
   'ClusterAvailabilityStatus': 'Available',
   'MasterUsername': 'awsuser',
   'DBName': 'myfirstdb',
   'Endpoint': {'Address': 'my-first-redshift.cbf32tbearcy.us-east-2.redshift.amazonaws.com',
    'Port': 5439},
   'ClusterCreateTime': datetime.datetime(2024, 2, 13, 5, 17, 43, 44000, tzinfo=tzutc()),
   'AutomatedSnapshotRetentionPeriod': 1,
   'ManualSnapshotRetentionPeriod': -1,
   'ClusterSecurityGroups': [],
   'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0aaf6ed6a8ca3224e',
     'Status': 'active'}],
   'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
     'ParameterApplyStatus': 'in-sync'}],
   'ClusterSubnetGroupName': 'default',
   'VpcId': 'vpc-0d6e2c48a8c676f0c',
   'AvailabilityZone': 'us-east-2a',
   'PreferredMaintenanceWindow': 'sat:05:30-sat:06:00',
   'PendingModifiedValues': {},
   'ClusterVersion': '1.0',
   'AllowVersi

In [13]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [14]:
myClusterProps

{'ClusterIdentifier': 'my-first-redshift',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'available',
 'ClusterAvailabilityStatus': 'Available',
 'MasterUsername': 'awsuser',
 'DBName': 'myfirstdb',
 'Endpoint': {'Address': 'my-first-redshift.cbf32tbearcy.us-east-2.redshift.amazonaws.com',
  'Port': 5439},
 'ClusterCreateTime': datetime.datetime(2024, 2, 13, 5, 17, 43, 44000, tzinfo=tzutc()),
 'AutomatedSnapshotRetentionPeriod': 1,
 'ManualSnapshotRetentionPeriod': -1,
 'ClusterSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0aaf6ed6a8ca3224e',
   'Status': 'active'}],
 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
   'ParameterApplyStatus': 'in-sync'}],
 'ClusterSubnetGroupName': 'default',
 'VpcId': 'vpc-0d6e2c48a8c676f0c',
 'AvailabilityZone': 'us-east-2a',
 'PreferredMaintenanceWindow': 'sat:05:30-sat:06:00',
 'PendingModifiedValues': {},
 'ClusterVersion': '1.0',
 'AllowVersionUpgrade': True,
 'NumberOfNodes': 1,
 'PubliclyAccessibl

In [15]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_NAME = myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

# Attach VPC and Secuity group to Redshift Cluster

In [16]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-04e835f185d636ad5')


# Connect to Redshift Cluster

In [23]:
!pip install psycopg2

In [30]:

import psycopg2
try:
    conn = psycopg2.connect(host=DWH_ENDPOINT, dbname=DB_NAME, user=DB_USER, password="Password123", port=5439)
except Exception as e:
    print("Error Could not make connection to the postgres DB")
    print(e)

conn.set_session(autocommit=True)

In [31]:
try:
    cur = conn.cursor()
except Exception as e:
    print("Error Could not get cursor to the DB")
    print(e)

# Create Data Models in Redshift

In [32]:
try:
    cur.execute("""
    create table users(
        userid integer not null distkey sortkey,
        username char(8),
        firstname varchar(30),
        lastname varchar(30),
        city varchar(30),
        state char(2),
        email varchar(100),
        phone char(14),
        likesports boolean,
        liketheatre boolean,
        likeconcerts boolean,
        likejazz boolean,
        likeclassical boolean,
        likeopera boolean,
        likerock boolean,
        likevegas boolean,
        likebroadway boolean,
        likemusicals boolean
    )
    """)
except psycopg2.Error as e:
    print("Error: Issue creating the Table")
    print(e)

In [33]:
try:
    cur.execute("""
    create table venue(
        venueid smallint not null distkey sortkey,
        venuename varchar(100),
        venuecity varchar(30),
        venuestate char(2),
        venueseats integer
    )
    """)
except psycopg2.Error as e:
    print("Error: Issue creating the Table")
    print(e)

In [35]:
try:
    cur.execute("""
    create table category(
        catid smallint not null distkey sortkey,
        catgroup varchar(30),
        catname varchar(30),
        catdesc varchar(50)
    );
    
    create table date(
        dateid smallint not null distkey sortkey,
        caldate date not null,
        day character(3) not null,
        week smallint not null,
        month character(5) not null,
        qtr character(5) not null,
        year smallint not null,
        holiday boolean default('N')
    );
    
    create table event(
        eventid smallint not null distkey sortkey,
        venueid smallint not null,
        catid smallint not null,
        dateid smallint not null,
        eventname varchar(200),
        starttime timestamp
    );
    
    create table listing(
        listid smallint not null distkey,
        sellerid integer not null,
        eventid integer not null,
        dateid smallint not null sortkey,
        numtickets smallint not null,
        priceperticket decimal(8,2),
        totalprice decimal(8,2),
        listtime timestamp
    );
    """)
except psycopg2.Error as e:
    print("Error: Issue creating the Table")
    print(e)

# Copy Data from S3 to Redshift using COPY command

In [37]:
try:
    cur.execute("""
        COPY users from 's3://tickit-items/allusers_pipe.txt' 
        credentials 'aws_iam_role=arn:aws:iam::992382696383:role/redshift-s3-access'
        delimiter '|'
        region 'us-east-2'
        
    """)
except psycopg2.Error as e:
    print("Error: Issue Copying the data from s3 to redshift tables")
    print(e)

In [38]:
try:
    cur.execute("""
        select * from users;        
    """)
except psycopg2.Error as e:
    print("Error: Issue fetching data from table")
    print(e)

In [41]:
row = cur.fetchone()
print(row)

(7, 'OWY35QYB', 'Tamekah', 'Juarez', 'Moultrie', 'WV', 'elementum@semperpretiumneque.ca', '(297) 875-7247', None, None, None, True, True, False, None, None, False, False)


# Close DB Connection and delete redshift Cluster

In [42]:
try:
    conn.close()
except psycopg2.Error as e:
    print(e)   

In [44]:
redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'my-first-redshift',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'awsuser',
  'DBName': 'myfirstdb',
  'Endpoint': {'Address': 'my-first-redshift.cbf32tbearcy.us-east-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2024, 2, 13, 5, 17, 43, 44000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0aaf6ed6a8ca3224e',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-0d6e2c48a8c676f0c',
  'AvailabilityZone': 'us-east-2a',
  'PreferredMaintenanceWindow': 'sat:05:30-sat:06:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'Numb